In [70]:
def evaluate_2(res, frac):
    assert len(res) == 60
    score = 1/3 + frac/3
    rcount = 0
    pcount = 0
    scount = 0
    
    allcount = 1
    if res[0] == "R":
        pcount += 1
    if res[0] == "S":
        rcount += 1
    if res[0] == "P":
        scount += 1

    for x in res[1:]:
        if x == "R":
            score += (scount/allcount) + (rcount/allcount)*frac
            pcount += 1
        if x == "S":
            score += (pcount/allcount) + (scount/allcount)*frac
            rcount += 1
        if x == "P":
            score += (rcount/allcount) + (pcount/allcount)*frac
            scount += 1
        allcount += 1
    
    return score

In [130]:
r10 = evaluate_2("RPS"*20, 1)
r10

41.64047105353967

In [131]:
r05 = evaluate_2("R"*9 +"S"*24+"P"*27, 0.5)
r05

29.799936172398098

In [132]:
r01 = evaluate_2("R"*7 +"S"*26+"P"*27, 0.1)
r01

27.771851342378618

In [133]:
r00 = evaluate_2("R"*6+"S"*26+"P"*28, 0)
r00

27.331726828014784

In [134]:
500*(r10 + r05 + r01 + r00)/4

15817.998174541397

In [135]:
r10, r05, r01, r00

(41.64047105353967, 29.799936172398098, 27.771851342378618, 27.331726828014784)

In [124]:
best = 0
best_idx = 0
for i in range(60):
    for j in range(60-i):
        k = 60-i-j
        r00 = evaluate_2("R"*i+"S"*j+"P"*k, 1)
        if r00 > best:
            best_idx = (i,j,k)
            best = r00

In [125]:
best_idx

(11, 20, 29)